# Introduction to `geopandas`
---

Today we are going to learn how to use python and Jupyter notebooks to learn the basics of working with geospatial data in `geopandas`. `geopandas` is built on top of the `pandas` package you saw in the previous lab. Many of the methods you will see in this lab are shared between the two packages.

In [ ]:
# This line tells iPython to not display warnings.
import warnings
warnings.filterwarnings('ignore')

# RUN THIS CELL FIRST or the notebook won't work
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import gtfs_kit as gk
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from IPython.display import display
import matplotlib.pyplot as plt

# These help the maps display nicely in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [30, 20]

First, let's load our data and see what kind of data we are working with.  The `read_file` method requires that we pass the _filepath_ from our current directory (the location of this notebook) to the data. The `pwd()` function shows you the your current location in the file system. Another way you can say this is that it __p__rints your __w__orking __d__irectory.

In [ ]:
pwd()

To get to the data, we would first enter the `lct_000b21a_e` folder from the working directory. The way we communicate this to the function is by passing `'lct_000b21a_e/lct_000b21a_e.shp'` as an argument. This folder contains geographic information organized by census tract for the entirety of Canada.

In [ ]:
canada_shp = gpd.read_file('lct_000b21a_e/lct_000b21a_e.shp')

Let's take a look at what our data looks like in Jupyter. We can use the `.head()` method to show the first 5 rows of our data. Similarly, you can use the `.tail()` method to show the last 5 rows of the data.

In [ ]:
canada_shp.head()

This is nice, but we'd like to have some data to go with these polygons, and we'd like to just work within the Census Metropolitan Area of Toronto. Let's load some data from an earlier lab.

In [ ]:
toronto_cma = pd.read_csv("~/git/cp101.github.io/labs/lab10/census21_data.csv")

In [ ]:
toronto_cma.columns

## Important

Many times over the course of reading and writing data from external sources, you might find that data types become corrupted. We can see that the GeoUID field in this dataframe ought to be right padded with zeroes so that it can match the expected form of the census tract ID in the geometry shapefile. Let's take care of that right away.

In [ ]:
# fix missingness and data types
toronto_cma = toronto_cma.fillna(0)
toronto_cma = toronto_cma.replace({'NA': 0})
toronto_cma = toronto_cma.replace({'': 0})
toronto_cma.iloc[:,4:] = toronto_cma.iloc[:,4:].apply(pd.to_numeric)
# pad to get correct geouid length
toronto_cma["GeoUID"] = toronto_cma["GeoUID"].astype(str).str.ljust(10, "0")

In [ ]:
toronto_cma.head()

Much better. Now, let's do a left inner join to the GeoPandas dataframes so that the resulting object will retain its geometric properties.

In [ ]:
toronto_gpd = canada_shp.merge(toronto_cma, how = "inner", left_on = "CTUID", right_on = "GeoUID")
toronto_gpd.head()

## Examining the data 

### Accessing the data <font color='red'> 

The `.loc[]` and `.iloc[]` methods allow us to view cells in a `DataFrame` or `GeoDataFrame` based on their name or location. The __i__ in `.iloc[]` stands for the __integer__ position of a cell, and accesses cells by location in index coordinates. The `.loc[]` method allows you to access cells by the index of the rows and the names of the columns. For both `.loc[]` and `.iloc[]`, the first argument refers to the row, and the second argument refers to the column.

Typically the row index will be the same as its integer position, but that is not always the case. Let's set up a `DataFrame` to see this in action.

In [ ]:
df = pd.DataFrame(data = {'a': [1, 2, 3],
                         'b': [4, 5, 6],
                         'c': [7, 8, 9],
                         'd': [10, 11, 12]},
                 index = [1.1, 1.2, 1.3])
df

If we want to access the number 8 from `df`, we would need to tell `.iloc[]` to look in `df` at row `1`, column `2` (remember that python starts counting from zero!).

In [ ]:
df.iloc[1, 2]

Using `.loc[]`, we would need to tell the method that we want row index `1.2`, column name `'c'`.

In [ ]:
df.loc[1.2, 'c']

You can also specify a range of indices to both of these methods if you want to access multiple adjacent cells. For `iloc[]`, this range will always refer to integer coordinates of the cells.

In [ ]:
# This code tells .iloc that we want rows 0 and 2, then all columns with index
# greater than or equal to 1.
df.iloc[[0, 2], 1:]

We can also refer to a range named columns and rows using `.loc[]`

In [ ]:
# This code tells .loc that we want row 1.2, columns 'b' through 'd' inclusive.
df.loc[[1.2], 'b':'d']

### Basic maps <font color='red'>

The `geometry` column contains a new data type called a polygon, which is how `geopandas` is able to store geographic information. Let's look at one of these polygons using `.loc[]`.

Is this a part of Toronto you recognize? Perhaps on a large park on the lakefront?

In [ ]:
toronto_gpd.loc[7, 'geometry']

Or perhaps this warped rectangle? A familiar university campus, near an urban park? 

In [ ]:
toronto_gpd.loc[55, 'geometry']

To view all of the polygons in the `GeoDataFrame`, we can just use the `.plot()` method. The documentation for this function can be found [here](http://geopandas.org/mapping.html).

In [ ]:
toronto_gpd.plot()

You can improve the appearance of this graph by removing the axis labels surrounding the map and adding a title. You may also see a line that says something like `<matplotlib.axes._subplots.AxesSubplot at [hexadecimal]>` above the map. You can prevent Jupyter from displaying this line by adding a `;` to the last line of code creating the map.

In [ ]:
toronto_gpd.plot()

# Adding the fontsize argument allows you to manipulate the font size.
plt.title('Toronto CMA Census Tracts', fontsize = 30)

# This turns the plot axes off.
plt.axis('off');

# Creating a new shapefile
    
These are nice maps, but there are ways to make it more informative. For starters, the Toronto CMA is quite a large area with a high amount of variance in census tract area. This does not make for the most visually pleasing map, so for now let's continue on a previous theme and focus just on the City of Toronto. 
    
We can do this by performing an inner join on the GeoUIDs of census tracts we queried in previous labs. 

In [ ]:
toronto_csd = pd.read_csv('~/git/cp101.github.io/labs/lab03/census21_data.csv')
# fix missingness and data types
toronto_csd = toronto_csd.fillna(0)
toronto_csd = toronto_csd.replace({'NA': 0})
toronto_csd = toronto_csd.replace({'': 0})
# create toronto census tracts from this column
toronto_census_tracts = toronto_csd["GeoUID"] 

In [ ]:
# a familiar number of census tracts!
toronto_census_tracts.shape

Let's try something a little different from our previous method of changing GeoUIDs to strings- a list comprehension. 

In [ ]:
toronto_census_tracts_str = [str(x).ljust(10, "0") for x in toronto_census_tracts]

If we were going to write this list comprehension as a for loop, we would have to create a new list to save our converted data into. It would look a little something like this:
```
toronto_census_tracts_str = []
for x in toronto_census_tracts:
    toronto_census_tracts_str.append(str(x).ljust(10, "0"))
```
List comprehensions are so much more efficient!

Next, we are going to use `isin` to subset the rows of `toronto_gpd` to just the rows belonging to City of Toronto census tracts and create a new `GeoDataFrame`.

In [ ]:
# Create a new GeoDataFrame
toronto = toronto_gpd[toronto_gpd['GeoUID'].isin(toronto_census_tracts_str)]

In [ ]:
# this should be True
toronto.shape[0] == len(toronto_census_tracts)

Next, we can save our new `GeoDataFrame`. The following code saves `toronto` as a file type called a "shapefile," which happens to be the same file type as the data we read in earlier. `geopandas` can also read and write many other geospatial file types, but we are just going to be using the same file type as before for now.

In [ ]:
toronto.to_file(driver='ESRI Shapefile', filename = 'shapefiles/toronto')

Now, we've been working with lengthy variable names for the last few labs. Let's use dictionaries to create some more user-friendly variables to type without having to sacrifice on well-formatted descriptions. One thing to make note of is that there is a typo in the CensusMapper API and that the vector `v_CA21_389: Average age` should really be `v_CA21_389: Median age`. APIs can be very useful, but always [check again](https://censusmapper.ca/api#api_variable) if something looks unusual.

In [ ]:
toronto.columns

In [ ]:
var_transforms = {'v_CA21_389: Average age':'medianage', 
                  'v_CA21_251: 65 years and over':'seniors',
                  'v_CA21_8: Total - Age':'tot_age',
                  'v_CA21_1040: Prevalence of low income based on the Low-income measure, after tax (LIM-AT) (%)':'lim_at',
                  'v_CA21_1085: Prevalence of low income based on the Low-income cut-offs, after tax (LICO-AT) (%)':'lico_at',
                  'v_CA21_1140: Gini index on adjusted household total income':'gini_total',
                  'v_CA21_1141: Gini index on adjusted household market income':'gini_market',
                  'v_CA21_1142: Gini index on adjusted household after-tax income':'gini_after_tax'
                 }

toronto = toronto.rename(columns = var_transforms)

toronto['pct_seniors'] = toronto['seniors'] / toronto['tot_age']

plot_titles = {'medianage':'Median age by census tract', 
               'pct_seniors':'Percent of population 65 years and over by census tract',
               'lim_at':'Prevalence of low income based on the LIM-AT (%) by census tract',
               'lico_at':'Prevalence of low income based on the LICO-AT (%) by census tract',
               'gini_total':'Gini index on adjusted household total income by census tract',
               'gini_market':'Gini index on adjusted household market income by census tract',
               'gini_after_tax':'Gini index on adjusted household after-tax income by census tract'
                 }

In [ ]:
toronto.columns

Now we can make a map with our new data. In fact, we can make a lot of maps with our new data. So let's make a function to avoid rewriting the same code over and over.

In [ ]:
def simple_plot(plot_var):
    toronto.plot(column = plot_var, legend = True)
    plt.title(plot_titles[plot_var], fontsize = 30)
    plt.axis('off');

In [ ]:
simple_plot('medianage')

Can you tell which census tracts are likely to be near the University of Toronto campus, just from looking at this map? If yes, why might that be? If no, what other information would you need in order to do so?

YOUR ANSWER HERE

If we want to make this map more informative, we can incorporate some of the other columns from the data into the map. For example, the `v_CA21_1142` vector contains [Gini index](https://www.physics.ucla.edu/~chester/GINI/index.html) of each census tract.

We can pass the column name as an optional argument to the `.plot()` method to create a map with a color gradient based on the values in this column. Setting `legend` equal to `True` in the function call will also tell the method that we want our final plot to have a legend so that we can interpret the colors on the map. 

In [ ]:
simple_plot('gini_after_tax')

If you want to change the color scheme of the map, you can choose the "color map" of the plot by adding in the optional `cmap` argument to `plot()`. You can choose from the variety of color maps available in the `matplotlib` package, which is what our plotting software is built off of. Above, we saw the default colormap, which is called viridis. If you want to view all the available colormaps, you can reference the documentation [here](https://matplotlib.org/users/colormaps.html).

In [ ]:
toronto.plot(column = 'gini_after_tax', legend = True, cmap = 'magma')
plt.title(plot_titles['gini_after_tax'], fontsize = 30)
plt.axis('off');

Now it's your turn! In the following cell, try creating your own choropleth map using a different column from `toronto_gpd`. If you want to see all of the columns in our data, you can scroll up to where we displayed all of the column names.

<font color = 'red'>

In [ ]:
# YOUR CODE HERE
toronto.plot(column = 'lico_at', legend = True, cmap = 'Wistia')
plt.title(plot_titles['lico_at'], fontsize = 30)
plt.axis('off');

What are some conclusions that you can draw from the map you created?

YOUR ANSWER HERE

## Coordinate Reference Systems (CRS)

Statistics Canada uses a special Coordinate Reference System called [Statistics Canada Lambert](https://epsg.io/3347) (EPSG:3347) for its shapefiles. You can read Statistics Canada's Illustrated Glossary page, including why this projection is used for most statistical analysis of Canadian geographies [here](https://www150.statcan.gc.ca/n1/pub/92-195-x/2021001/other-autre/coord/coord-eng.htm). 


A standard CRS used internationally for a variety of catography and satellite navigateion is the World Geodetic System. The Map and Data Library has several Toronto-centric datasets- pay special attention to their projection before working with them. The EPSG values are shortened abbreviations of the coordinate system names that can be used to easily convert between projections using geospatial software/libraries. You can read more about EPSG [here](https://epsg.org/home.html).

In [ ]:
toronto.crs

In [ ]:
# reprojects coordinates to new coordinate reference system
toronto = toronto.to_crs(4326)

In [ ]:
toronto.plot()
plt.title('City of Toronto Census Tracts', fontsize = 30)
plt.axis('off');

# More geospatial data types in `geopandas`

So far, we have only been creating maps using polygons, but `geopandas` has a few more data types we can work with. First, we need some new data to work with. In the `ttc` folder, we have folders named `bart_stations` and `bart_routes` containing geospatial data about the BART system. Load in the data as we did with `alameda` file from the beginning of the lab.

### Working with GTFS data


In [ ]:
""" Utility functions for GGR337 CKAN API lab.

These functions have been adapted from Data 100 course materials.
"""

def retrieve_package(keys):
    """Downloads package info of a dataset using CKAN's API

    Args:
        keys (dict): A Python dictionary with Toronto Open Dataset id 
          keys (strings), like this (but filled in):
            {
                "id": "<dataset id here>",
            }

    Returns:
        list: A list of Dictonary objects, each representing help, response, and result."""
    import requests
    # Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
    # https://docs.ckan.org/en/latest/api/
    base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
    # Datasets are called "packages". Each package can contain many "resources"
    # To retrieve the metadata for this package and its resources, use the package name in this page's URL:
    url = base_url + "/api/3/action/package_show"
    package = requests.get(url, params = keys).json()
    return package


def load_tpl_events(method):
    """Loads the dataframe of the toronto public library events feeding using the predownloaded csv or API request
    
    Args:
        method (string): can either be 'read_csv' for predownloaded csv or 'API' for API request
        """
        
    if method == "read_csv":
        return pd.read_csv("tpl-events-feed.csv")
    elif method == "API":
        
        package = retrieve_package("library-branch-programs-and-events-feed")
        
        
        for idx, resource in enumerate(package["result"]["resources"]):
            if resource["datastore_active"]:
                # to get all records in CSV format
                url = base_url + "/datastore/dump/" + resource["id"]
                # do a GET request on the url and access its text attribute
                resource_dump_data = requests.get(url).text
                # read the raw csv text into a pandas dataframe to work with it
                return pd.read_csv(StringIO(resource_dump_data), sep=",")
    else:
        print("Unacceptable argument for 'method'. Use either 'read_csv' or 'API'.")
        return
              
def load_public_survey(method):
    """Loads the dataframe of the toronto public survey results using the predownloaded csv or API request
    
    Args:
        method (string): can either be 'read_csv' for predownloaded csv or 'API' for API request
        """
        
    if method == "read_csv":
        return pd.read_csv("torr-public-survey-results.csv")
    elif method == "API":
        
        package = retrieve_package({ "id": "toronto-office-of-recovery-and-rebuild-public-survey-results"})
        
        for idx, resource in enumerate(package["result"]["resources"]):
           # To get metadata for non datastore_active resources:
            if resource["format"] == "CSV":
                url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
                resource_data = requests.get(url).json()
                # do a GET request on the url and access its text attribute
                resource_dump_data = requests.get(resource_data['result']['url']).text
                # read the raw csv text into a pandas dataframe to work with it
                return pd.read_csv(StringIO(resource_dump_data), sep=",")
    else:
        print("Unacceptable argument for 'method'. Use either 'read_csv' or 'API'.")
        return

In [ ]:
# https://open.toronto.ca/dataset/ttc-routes-and-schedules/
# https://open.toronto.ca/dataset/major-city-wide-cycling-routes/

ttc_routes_package = retrieve_package({"id":"ttc-routes-and-schedules"})
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
for idx, resource in enumerate(ttc_routes_package["result"]["resources"]):
    # To get metadata for non datastore_active resources:
    if not resource["datastore_active"]:
        url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
        resource_data = requests.get(url).json()
zipurl = resource_data['result']['url']
zipresp = urlopen(zipurl)
# Create a new file on the hard drive
tempzip = open("ttc/tempfile.zip", "wb")
tempzip.write(zipresp.read())
# Close the newly-created file
tempzip.close()

In [ ]:
# a lot is going on here, but it is just reading in the contents of the zipped file to dataframes
# adapted from: https://max-coding.medium.com/how-to-process-gtfs-data-using-pandas-geopandas-4b34f2ad3273

with ZipFile("ttc/routes_schedules.zip") as myzip:
    shapes_df = pd.read_csv(myzip.open("shapes.txt"), dtype={
        'shape_id': 'str', 
        'shape_pt_lat': 'float', 
        'shape_pt_lon': 'float',  
        'shape_pt_sequence': 'Int64', 
        'shape_dist_traveled': 'float'
    })
    shapes_gdf = gpd.GeoDataFrame(shapes_df, 
        geometry=gpd.points_from_xy(shapes_df.shape_pt_lon, shapes_df.shape_pt_lat)).set_crs(epsg=4326)
    
    stops_df = pd.read_csv(myzip.open("stops.txt"), dtype={
        'stop_id': 'str', 
        'stop_code': 'str',
        'stop_name': 'str',
        'stop_lat': 'float',
        'stop_lon': 'float'
    })
    stops_gdf = gpd.GeoDataFrame(stops_df, 
        geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)).set_crs(epsg=4326)
    
    routes_df = pd.read_csv(myzip.open("routes.txt"), dtype={
        'route_id': 'str',  
        'agency_id': 'str',  
        'route_short_name': 'str',  
        'route_long_name': 'str', 
        'route_desc': 'str', 
        'route_type': 'Int64',
        'route_color': 'str',  
        'route_text_color': 'str'
    })
    
    trips_df = pd.read_csv(myzip.open("trips.txt"), dtype={
        'route_id': 'str', 
        'service_id': 'str',  
        'trip_id': 'str',
        'shape_id': 'str', 
        'trip_headsign': 'str', 
        'direction_id': 'str',  
        'block_id': 'str', 
        'wheelchair_accessible': 'str', 
        'bikes_allowed': 'str'
    })
    
    calendar_df = pd.read_csv(myzip.open("calendar.txt"), dtype={
        'service_id': 'str',  
        'monday': 'bool',  
        'tuesday': 'bool',  
        'wednesday': 'bool',  
        'thursday': 'bool',  
        'friday': 'bool', 
        'saturday': 'bool',  
        'sunday': 'bool',  
        'start_date': 'str', 
        'end_date': 'str',
    })
    
    calendar_dates_df = pd.read_csv(myzip.open("calendar_dates.txt"), dtype={
        'service_id': 'str',  
        'date': 'str',
        'exception_type': 'Int64',
    })
    

In [ ]:
shapes_df.head()

In [ ]:
segments_df.head()

In [ ]:
stops_df.head()

In [ ]:
routes_df.head()

In [ ]:
trips_df.head()

In [ ]:
calendar_df

From this table, we can see that there are 9 lines in the Toronto TTC file and quite a few of them haven't run over the period for which we have data. `service_id == 1` runs weekdays, `service_id == 2` runs Saturdays, and `service_id == 3` runs Sundays.

In [ ]:
calendar_dates_df

These dataframes are quite large and plotting maps of them can get quite messy, so let's pick one day of the week to focus on. You can change this to select any day of the week that you want, just be sure to type it in `lowercase`. 

In [ ]:
# defaults to whatever day of the week it is when you run this cell, but you can manually change it to whatever you like!
day_of_week_name = dt.datetime.today().strftime('%A').lower()
day_of_week_name

# TODO: explain the motivation for the next cell

In [ ]:
coords = shapes_df[["shape_pt_lat", "shape_pt_lon", "shape_pt_sequence"]]
coords_roll_1 = np.roll(coords, 1, axis=0)
segments = pd.DataFrame(np.concatenate([coords_roll_1, coords], axis=1), columns=["start_lat", "start_lng", "start_seq", "end_lat", "end_lng", "end_seq"])
segments_df = shapes_df[["shape_id"]].join(segments)
segments_df = segments_df[segments_df.end_seq != 1]
segments_df = segments_df.drop(columns=['end_seq']).rename(columns={ "start_seq": "seq" })
segments_df.head()

# TODO: explain motivation for the next cell

In [ ]:
shape_trips = trips_df.groupby(by="shape_id").size().to_frame("tot_trips")
shape_trips.head()

In [ ]:
shape_trips[shape_trips.index == "982620"]

# TODO: explain motivation for the next cell

In [ ]:
def limit_to_bounding_box(gdf, bounding_box):
    return gdf.cx[bounding_box["west"]:bounding_box["east"],bounding_box["south"]:bounding_box["north"]]


toronto_bb = {
    "north": min(toronto.bounds['maxy']),
    "south": min(toronto.bounds['miny']),
    "west": min(toronto.bounds['maxx']),
    "east": min(toronto.bounds['minx']),
}

In [ ]:
# want to join trip counts by shape_id to the segments
shape_segments_trips = pd.merge(segments_df, shape_trips, left_on="shape_id", right_index=True)
segment_trips = shape_segments_trips.groupby(by=["start_lat", "start_lng", "end_lat", "end_lng"]).sum("tot_trips").reset_index()
segment_trips.head()

# TODO: explain motivation for next cell

In [ ]:

from shapely.geometry import Point
from shapely.geometry import LineString

segment_trips['start_point'] = segment_trips.apply(lambda row: Point(row['start_lng'], row['start_lat']), axis = 1)
segment_trips['end_point'] = segment_trips.apply(lambda row: Point(row['end_lng'], row['end_lat']), axis = 1)

segment_trips['geometry'] = segment_trips.apply(lambda row: LineString([row['start_point'], row['end_point']]), axis = 1)

segment_trips.head()

In [ ]:
segment_trips_gpd = limit_to_bounding_box(gpd.GeoDataFrame(segment_trips), toronto_bb)
segment_trips_gpd.head()

First, we need the base map. We can use the aggregated `GeoDataFrame` we created earlier in this notebook. Try setting the `color` and `edgecolor` arguments so the map looks more uniform.

In [ ]:
base = toronto.plot()
plt.axis('off');

Next, overlay the routes and stations and plot the graph by using this as the base for the map. The Jupyter notebook will not remember the map you drew in the previous cell even if you assigned it a name, so make sure to plot the base map again in the following cell.

In [ ]:

segment_trips_gpd.plot(ax=base, color = '#789b73', linewidth = 6, zorder=2)
stops_gdf.plot(ax=base, color = '#cdfd02', markersize = 25, zorder=3)
plt.axis('off');

In [ ]:
fig = plt.figure(figsize=(30,21))
base = toronto.plot(color = '#d8dcd6', zorder=1)
segment_trips_gpd.plot(cmap="plasma", column='tot_trips', legend=True)
plt.title(f"Number of trips on TTC", fontsize=30)

Congratulations! You're done with this lab! If you are interested in learning more about what you can do with `geopandas`, you can find the documentation for the package [here](http://geopandas.org/reference.html).

***

### Adapted from Berkeley lab content created by Monica Wilkinson and Vera Wang
### References:
- http://geopandas.org/mapping.html
- https://matplotlib.org/users/colormaps.html